## Imports

In [ ]:
import sys 
import numpy as np 
import matplotlib.pyplot as plt


import torch
import torchvision 
import torch.optim as optim
import torch.nn as nn

sys.path.insert(0, '../../Utils')

import models
from train import *
from metrics import * 

print("Python: %s" % sys.version)
print("Pytorch: %s" % torch.__version__)

# determine device to run network on (runs on gpu if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



## Hyperparameters

In [ ]:
n_epochs = 30
batch_size = 128
lr = 0.0001
criterion = nn.CrossEntropyLoss()

## Load CIFAR10

In [ ]:
# define series of transforms to pre process images 
transform = torchvision.transforms.Compose([
    torchvision.transforms.Pad(2),
    torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
    

classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


# load training set 
cifar10_trainset = torchvision.datasets.CIFAR10('../../Datasets/', train=True, transform=transform, download=True)
cifar10_trainloader = torch.utils.data.DataLoader(cifar10_trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# load test set 
cifar10_testset = torchvision.datasets.CIFAR10('../../Datasets/', train=False, transform=transform, download=True)
cifar10_testloader = torch.utils.data.DataLoader(cifar10_testset, batch_size=32, shuffle=False, num_workers=2)

# helper function to unnormalize and plot image 
def imshow(img):
    img = np.array(img)
    img = img / 2 + 0.5
    img = np.moveaxis(img, 0, -1)
    plt.imshow(img)
    
# display sample from dataset 
imgs,labels = iter(cifar10_trainloader).next()
imshow(torchvision.utils.make_grid(imgs))  

## Train Resnet18

In [ ]:
# load the torchvision resnet18 implementation 
resnet18 = torchvision.models.resnet18(num_classes=10).to(device)

# resnet fix for cifar10 image size 
resnet18.avgpool = nn.AvgPool2d(2, stride=1)

resnet18.apply(models.weights_init)

#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
resnet18_optim = optim.Adam(resnet18.parameters(), lr=lr)

resnet18 = resnet18.to(device)
train(resnet18, cifar10_trainloader, cifar10_testloader, resnet18_optim, criterion, n_epochs, classes=classes, verbose=False)

print("\nPerformance on training set: ")
train_accuracy = eval_target_net(resnet18, cifar10_trainloader, classes=None)

print("\nPerformance on test set: ")
test_accuracy = eval_target_net(resnet18, cifar10_testloader, classes=None)

## Train VGG16

In [ ]:
vgg16 = torchvision.models.vgg16(num_classes=10).to(device)
# vgg16 fix for cifar10 image size 
vgg16.classifier = nn.Sequential(
            nn.Linear(512, 64),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(64, 64),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(64, 10),
        )

vgg16.apply(models.weights_init)

vgg16_optim = optim.Adam(vgg16.parameters(), lr=lr)

vgg16 = vgg16.to(device)
train(vgg16, cifar10_trainloader, cifar10_testloader, vgg16_optim, criterion, n_epochs, classes=classes, verbose=False)

print("\nPerformance on training set: ")
train_accuracy = eval_target_net(vgg16, cifar10_trainloader, classes=None)

print("\nPerformance on test set: ")
test_accuracy = eval_target_net(vgg16, cifar10_testloader, classes=None)

## Train AlexNet

In [ ]:
alexnet = models.AlexNet(n_classes=10).to(device)

alexnet.apply(models.weights_init)

alexnet_optim = optim.Adam(alexnet.parameters(), lr=lr)

train(alexnet, cifar10_trainloader, cifar10_testloader, alexnet_optim, criterion, n_epochs, classes=classes, verbose=False)

print("\nPerformance on training set: ")
train_accuracy = eval_target_net(alexnet, cifar10_trainloader, classes=None)

print("\nPerformance on test set: ")
test_accuracy = eval_target_net(alexnet, cifar10_testloader, classes=None)

In [ ]:


#squeezenet = torchvision.models.squeezenet1_0()
#densenet = torchvision.models.densenet161()
#inception = torchvision.models.inception_v3(num_classes=10)

